In [1]:
# %%bash
import os
os.environ['http_proxy'] = 'http://localhost:8081'
os.environ['https_proxy'] = 'http://localhost:8081'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,3'

In [2]:
%cd '/mnt/nfs-data/users/anhvth5/projects/guidance/'

/mnt/nfs-data/users/anhvth5/projects/guidance


In [3]:
from speedy import *
from guidance import *

In [22]:
data = load_by_ext('../tvpl_crawler/data/tvpl_qa_short_answer_giaothong/qa_short_answer/*')
questions = []
for qa in data:
    questions.extend(qa['questions'])

2023-12-30 04:31:12.414 | INFO     | speedy.utils:multi_process:261 - Multi processing load_by_ext | Num samples: 6788


  0%|          | 0/6788 [00:00<?, ?it/s]

In [48]:
qwen = models.get_qwen_guidance(device_map=0, do_update_lm_head=True)

2023-12-30 04:52:05.778 | INFO     | guidance.models.transformers._qwen:get_qwen_guidance:238 - Load Qwen from "/public-llm/Qwen-72B-Chat-Int4/" with device_map "0"


2023-12-30 04:52:06.173 | INFO     | guidance.models.transformers._qwen:__init__:47 - Set tokenizer.eos_token_id = tokenizer.eod_id


len(tokenizer)=151851


In [33]:
import requests

def search(query, k=3, port=19001):
    url = f"http://localhost:{port}/api/search"
    params = {
        "query": query,
        "return_dict": "true",
        'k': k
    }
    response = requests.get(url, params=params, proxies={'http':'', 'https':''})
    topk = response.json()['topk']
    print(topk[0].keys())
    docid_to_text = {t['pid']:f'{t["title"]}| {t["text"]}' for t in topk}
    return docid_to_text


In [35]:
# from IPython.display import clear_output

# query = 'Thủ tục đăng ký biển số xe máy mới nhất'
# search_results = search(query, k=3)
# doc_ids = list(search_results.keys())

# with system():
#     lm = qwen + 'You are now tasked to answer user questions based on the retrieved documents\nResponse language: Vietnamese'

# with user():
#     lm += '''Query: {}
#     Retrieved documents: {}
    
#     - Hãy trả về danh sách tài liệu liên quan nhất cho câu hỏi này
#     - Danh sách các tài liệu cần có thứ tự ưu tiên với tài liệu có độ liên quan cao nhất ở vị trí đầu tiên
#     - Fomat: [id1, id2...]
#     - Nếu không có hãy trả về []
#     '''.format(query,json.dumps(search_results, indent=4, ensure_ascii=False))
    
# with assistant():
#     lm += f'Câu hỏi: {query}'
#     lm += f'\nDanh sách các tài liệu liên quan: ['+gen(max_tokens=10000, name='relevant_docs', stop=']')+']'
    
# import json
# relevant_docs = {id:search_results[id] for id in eval('['+lm['relevant_docs']+']')}

'<|im_start|>system\nYou are now tasked to answer user questions based on the retrieved documents\nResponse language: Vietnamese<|im_end|>\n<|im_start|>user\nQuery: Mẫu sổ theo dõi trong xử phạt vi phạm hành chính ngành giao thông vận tải là những mẫu nào?\nRetrieved documents: {\n  "105499": "Thông tư 30/2022/TT-BGTVT biểu mẫu xử phạt vi phạm hành chính ngành Giao thông vận tải mới nhất |  Sổ theo dõi biểu mẫu xử phạt vi phạm hành chính trong lĩnh vực giao thông vận tải\\n2\\nMS02\\nSổ theo dõi xử phạt vi phạm hành chính\\n3\\nMS03\\nSổ theo dõi tạm giữ, giữ, trả lại tang vật, phương tiện, giấy phép, chứng chỉ hành nghề bị tạm giữ, giữ\\n\\nMẫu sổ 01\\nSỔ THEO DÕI BIỂU MẪU XỬ PHẠT VI PHẠM HÀNH CHÍNH TRONG NGÀNH GIAO THÔNG VẬN TẢI (*)\\nTT\\nNhập biểu mẫu\\nCấp phát biểu mẫu\\nTheo dõi sử dụng biểu mẫu\\nNgày nhập\\nSố lượng quyển\\nSố lượng tờ/số seri\\nKý nhận (ký, ghi rõ họ tên người nhận)\\nNgày cấp\\nSố lượng quyển\\nSố lượng tờ/số seri\\nĐơn vị (đội, bộ phận) nhận\\nKý nhận (ký, 

In [42]:
import guidance


@guidance
def giaothong_rag(lm, query, gt_answer=None, k=None, db='qa'):
    port = 19003 if db == 'qa' else 19001
    if k is None:
        k = 3 if db == 'qa' else 5
    relevant_docs = search(query, k=k, port=port)

    with system():
        lm = (
            lm
            + "You are now tasked to answer user questions based on the retrieved documents\nResponse language: Vietnamese"
        )

    with user():
        lm += """Query: {}
        Retrieved documents: {}
        Guide:
            - Hãy phân tích sau đó đưa ra kết luận về câu hỏi này
            - 
            - Response language: Vietnamese
        """.format(
            query, json.dumps(relevant_docs, ensure_ascii=False, indent=2)
        )

    with assistant():
        lm += f"**Câu hỏi** {query}"
        lm += "\n\n**Suy luận**" + gen(max_tokens=2000, name='reasoning',stop="**")
        lm += "\n\n**Kết luận**" + gen(
            max_tokens=2000, name="conclusion", stop="<|im_end|>"
        )
    return lm

In [73]:
qa = questions[11]
lm = qwen+giaothong_rag(query = qa['question'], db='docs')

In [75]:
@guidance
def compare_with_answer(lm, question, conclusion, answer):
    with system():
        lm = lm + '''You are now task to do the NLI task. You are given a question, a conclusion and an answer. 
        Your task is to compare the conclusion with the answer and give the correct answer. 
        Response with a single word: "correct" if the conclusion aligns with the answer, "incorrect" otherwise.'''
    with user():
        lm += f"**Question** {question}"
        lm += f"\n\n**Conclusion** {conclusion}"
        lm += f"\n\n**Answer** {answer}"
    with assistant():
        lm += "The conclusion is " + select(["correct", "incorrect"], name="correctness")
    return lm

compare_lm = qwen+compare_with_answer(qa['question'], lm['conclusion'], qa['short_answer'])